# Домашнее задание по transfer learning

* Тема: выполнить transfer learning + fine-tuning для модели на собранном датасете. Примеры датасетов: киты и акулы, снятых под водой; экзотические птицы; фрукты и овощи; и так далее.
* Работа ведется на tensorflow. За основу можно взять готовые материалы или ноутбуки: [tf](https://colab.research.google.com/drive/18N3JWFS7v2MB_9owoLpXZE_NS8orGo2h?usp=sharing), [keras.io](https://keras.io/guides/transfer_learning/), (если ссылки не работают, напишитe в slack).
* Результатом работы должен быть обученный классификатор:
  * Сама модель и графики ее обучения
  * Пример работы на нескольких изображениях
  * Ноутбук с кодом обучения

# Более подробные инструкции
* Выполнять д/з можно в ноутбуках, скриптах .py, colab - где удобнее.
* Для сборки датасета используйте [fatkun](https://chrome.google.com/webstore/detail/fatkun-batch-download-ima/nnjjahlikiabnchcpehcpkdeckfgnohf?hl=en). Работает, к сожалению, только в Google Chrome. Можно использовать и другие инструменты (js/python скрипты), но времени потратите больше, скорее всего.
* Для начала сборки датасета попробуйте два запроса: whale underwater, shark underwater. Начните с нескольких сотен изображений (подготовка займет ~40 минут), обучите сеть на них. (*Если у вас мало времени/другие сложности со сбором данных, можете взять готовый датасет, инструкция в конце ноутбука. Но советую попробовать собрать свой*)
* Не нужно чрезмерных усилий, хватит ~100-200 картинок на каждый класс для старта. Не забывайте про базовую аугментацию.
* Не забудьте сделать train, validation, test - сплиты! Тестовый сплит понадобится для честной model selection (напоминаю, что early stopping - это тоже техника model selection).
* Задание:
  * Подготовьте данные. Поскольку дальше будут использоваться параметры ImageNet, рекомендую приводить картинки к формату (224, 224) и использовать такой же размер входного слоя сети.
  * Постройте t-SNE представление векторов, полученных feature extractor'ом. (можно поэкспериментировать с t-SNE, в частности, попробовать 3D-графики).
  * Поэкспериментируйте с разным количеством добавленных слоев классификатора и количеством нейронов в них. *Будет ли модель обучаться, и если да, то в каких условиях*, если добавить один полносвязный слой классификатора непосредственно к feature extractor'у? 
  * Обучите модель transfer learning, взяв за основу любую модель, обученную на ImageNet. Если у вас два класса, постарайтесь получить accuracy 0.75+ для сбалансированной тестовой выборки :) Можно больше.
  * После того, как модель будет обучена, выполните fine-tuning, разморозив основную часть весов модели. Постройте предствление t-SNE векторных представлений после fine-tuning'a и сравните с тем, которое было до него. Fine-tuning можно выполнять как с метками, так и при помощи triplet loss'a.
  * Также добавьте loss/accuracy графики для обучающих и валидационных данных.Можно сделать это в tensorboard.
  * *Дополнительно*: подумайте, как сочетание подхода с поиском векторных представлений и аугментаию можно использовать для обучения на данных с частичной или отсутсвующей разметкой. Интересно обсудить ваши идеи.
* Пришлите ссылку на ноутбук с кодом / скрипты как результаты работы.
* Если есть вопросы - пишите в slack.




# Код, который может понадобиться

In [ ]:
%tensorflow_version 2.x
%load_ext tensorboard
import os
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
from tensorflow import keras

from numpy.linalg import norm
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from PIL import Image
import io


from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from numpy.linalg import norm


def make_tsne_plot(vectors, labels):
  pca_dimension = 100
  if vectors.shape[1] <= pca_dimension * 2:
    pca_vectors = vectors
  else:
    pca = PCA(pca_dimension)
    pca.fit(vectors)
    pca_vectors = pca.transform(vectors)

  tsne_results = TSNE(n_components=2,
                      verbose=1,
                      metric='euclidean').fit_transform(pca_vectors) 

  cmap = plt.cm.get_cmap('coolwarm')
  plt.figure(figsize=(10, 8), dpi=80)
  scatter = plt.scatter(tsne_results[:,0],
                        tsne_results[:,1], 
                        c=labels, 
                        cmap=cmap)
  plt.colorbar(scatter)
  plt.show()


def make_extractor(img_shape, model=None, 
                   dropout_rate=0.25, trainable=False):
  if model is None:
    extractor = keras.applications.InceptionResNetV2(
        include_top=False, weights="imagenet")
  else:
    extractor = model(include_top=False, weights="imagenet")
  for layer in extractor.layers:
    layer.trainable = trainable

  gap = keras.layers.GlobalAveragePooling2D()
  dropout = keras.layers.Dropout(dropout_rate)

  input = keras.layers.Input(img_shape)
  x = extractor(input)
  x = dropout(x)
  x = gap(x)

  return keras.Model(input, x)


def make_triple_loss_model(img_shape, model=None, embedding_shape=32, 
                          units=64, dropout_rate=0.25, trainable=False): 
  """
  Создаем модель. По умолчанию - InceptionResNetV2.
  Если передан конструктор model, используется он (должен поддерживать 
  интерфейс keras.applications).
  
  """
  extractor = make_extractor(img_shape, model, 
                             dropout_rate=dropout_rate, 
                             trainable=trainable)

  dense_1 = keras.layers.Dense(units, activation='relu')
  dense_2 = keras.layers.Dense(units, activation='relu')
  embedding_layer = keras.layers.Dense(embedding_shape, activation='linear')
  dropout = keras.layers.Dropout(0.25)

  images = keras.layers.Input(img_shape, name='input_shape')
  labels = keras.layers.Input((1,), name='input_label')

  x = extractor(images)
  x = dropout(x)
  x = dense_1(x)
  x = dropout(x)
  x = dense_2(x)
  embeddings = embedding_layer(x)

  return keras.Model(inputs=images, outputs=embeddings)

## Обратите внимание!

`ImageDataGenerator` может сам разделять данные из одной директории на train и val.

In [ ]:

dataget = keras.preprocessing.image.ImageDataGenerator(
    zca_epsilon=1e-06,
    rotation_range=0.25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    shear_range=0.2,
    zoom_range=0.2,
    channel_shift_range=0.0,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=1/255.,
    validation_split=0.20  # <- Если выставлен этот параметр
)

Тогда при создании генераторов из `datatagen` вам будет нужно указать название сплита:

In [ ]:
dataset_path = None

In [ ]:
BATCH_SIZE = 64
IMG_SHAPE = (224, 224, 3)
train_generator = dataget.flow_from_directory(
    dataset_path, target_size=IMG_SHAPE[:-1],
    batch_size=BATCH_SIZE, subset="training")
val_generator = dataget.flow_from_directory(
    dataset_path, target_size=IMG_SHAPE[:-1],
    batch_size=BATCH_SIZE, subset="validation")
training_steps = int(np.ceil(train_generator.n // BATCH_SIZE))
val_steps = int(np.ceil(val_generator.n // BATCH_SIZE))

# Если вам нужен готовый датасет, можно попробовать:

In [ ]:
# thx https://gist.github.com/frogermcs/ed9fc359941efe54cc80d5b15f87bf77
import optparse

data_root = tf.keras.utils.get_file(
  'flower_photos',
  'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

dataset_path = os.path.join(os.path.expanduser('~'), 
                            '.keras/datasets/flower_photos/')

In [ ]:
dataset_path